# Selecting Temporal Range

We want to set up an interactive element to let the user quickly specify a range of dates to use as a filter on their data. The Jupyter environment has a few options for doing this.  We'll look at a couple and recommend the one that seems easiest on the user. 

In [ ]:
import ipywidgets
from ipywidgets import interact
import datetime
import pandas as pd


## Set Dates Using `DatePicker` widget
The easy option is to use a `DatePicker` widget to specify the start and end dates separately: 

In [ ]:
start_date = ipywidgets.DatePicker(
    description="Start",
    value=datetime.date(2020, 1, 1),
    disabled=False
)
end_date = ipywidgets.DatePicker(
    description="End",
    value=datetime.date(2022, 7, 1),
    disabled=False
)
display(start_date)
display(end_date)

In [ ]:
print(f"Range is from {start_date.value} to {end_date.value}")

Note that we set default values when each `DatePicker` was created. You don't have to do that, but the user experience is better if you do. The big reason is that the picker's `value` is `None` if you don't specify a start/default value.  Start with a good default, and we won't have to check the data for sanity later. 

The `DatePicker` is not configurable in terms of granularity.  It can't be restricted to pick a week, a month, or just a year. To do that, we need to use another tool (see below).  

A slight enhancement to the dual `DatePicker` approach is to combine them using `interact()`: 

In [ ]:
def fn(start, end):
    print(f"Range is from {start} to {end}")

_ = interact(fn, start=ipywidgets.DatePicker(value=datetime.date(2020, 1, 1)), end=ipywidgets.DatePicker(value=datetime.date(2020, 12, 31)))

The advantage here is that the plumbing for listening to events and setting handlers has all been done for us by `interact()`. The `fn()` function is called whenever either of the `DatePicker` objects updates its value.

Using `interact`, the selected value of each of those widgets is not available later, unless you take steps to ensure that behavior within your handler function. This makes `interact` a good choice if the dates will be used to directly modify other elements displayed from the same cell (as in the print statements, above). But if you want those values later, `interact` may not be the right way to go.   

## Set Dates Using `SelectionRangeSlider` widget

In [ ]:
dates = [datetime.date(2020, i, 1) for i in range(1, 13)]
options = [(i.strftime('%b, %Y'), i) for i in dates]
rSlider = ipywidgets.SelectionRangeSlider(
    options=options,
    index=(0, 11),
    description='Months:',
    disabled=False,
    layout={'width':'600px'}
)
display(rSlider)

In [ ]:
rSlider.value

... and you can also use `interact`, similar to what we did above with the `DatePicker`: 

In [ ]:
def fn(r):
    print(f"Range is from {r[0]} to {r[1]}")

_ = interact(fn, r=ipywidgets.SelectionRangeSlider(
    options=options,
    index=(0, 11),
    description='Months:',
    disabled=False,
    layout={'width':'600px'}
))

The value (and the challenge) of using this range slider is in setting up the tuples that establish the labels and values for the range of options on the slider. The above example set that up with: 
```python
dates = [datetime.date(2015, i, 1) for i in range(1, 13)]
options = [(i.strftime('%b:%y'), i) for i in dates]
``` 

In [ ]:
dates

In [ ]:
options

Setting up the `options` array of 2-tuples is a bit of a pain in the neck, but it lets us have a lot of control over how to display the options to the user. In this way, we can set up the sliders to just be years, months-years, or some other useful designation. See the `strftime()` method on dates to explore useful label formatting.  

Note that the actual pick-able values are stored as `datetime.date` in this case, which will be useful later, as xarray (and other tools) can extrapolate from these to the correct granularity to select data.  

## Customizing to the Dataset

The most useful widget would be pre-configured to the dataset it will be used to select. That is, we should examine the data in the dataset to come up with the minimum and maximum date values, then configure the list of available options from that.  We don't have to know the granularity of the data, since we will be storing the selection dates as `datetime.date` values. 

In [ ]:
# Mimic parsing the data to get min and max date values. 
mindate = ipywidgets.DatePicker(
    description="Start",
    value=datetime.date(2021, 1, 1),
    disabled=False
)
maxdate = ipywidgets.DatePicker(
    description="End",
    value=datetime.date(2022, 7, 1),
    disabled=False
)
frq = ipywidgets.Dropdown(
    description="Granularity:",
    options=[('Daily', 'D'), ('Weekly', 'W'), ('Monthly (start of month)', "MS"), ("Monthly (end of month)", "M")],
    value="MS",
    disabled=False
)
print("Simulating parsing the data.... set the min & max dates, along with the granularity you want in the slider: ")
display(mindate)
display(maxdate)
display(frq)

In [ ]:
# Using the built-in from pandas, because it does the job easily -- and we don't have to roll our own. 
print(f"Creating dates by {frq.value} between {mindate.value} and {maxdate.value}")
daterange = [i.date() for i in pd.date_range(start=mindate.value, end=maxdate.value, freq=frq.value).to_pydatetime()]
print(f"{len(daterange)} entries.")


Note carefully that I used a list comprehension to turn the list of `datetime` objects into plain old `date`.  If your data needs to have less-than-a-day granularity, then you can skip the comprehension and just use the `pd.date_range(...).to_pydatetime()` call. 

Now that we have an ordered list of `date` data, we can build the range using it. 

In [ ]:

slider_options = [(i.strftime('%b, %Y'), i) for i in daterange]
r = ipywidgets.SelectionRangeSlider(
    options=slider_options,
    index=(0, len(daterange)-1),
    description='Date range:',
    disabled=False,
    layout={'width':'800px'}
)
display(r)

In [ ]:
r.value

... Or... with `interact`:

In [ ]:
def fn(r):
    print(f"Selected Range is from {r[0]} to {r[1]}")

_ = interact(fn, r=ipywidgets.SelectionRangeSlider(
    options=slider_options,
    index=(0, len(daterange)-1),
    description='Date range:',
    disabled=False,
    layout={'width':'800px'}
))